In [126]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## define a fixed samplerate

In [127]:
samplerate=200

## load mass

In [128]:
annotations, signalsMetadata = loadMASSSpindles(masspath,forceSamplerate=samplerate)

In [129]:
annotations.head()

,type,expert,subjectId,labelerId,startTime,duration,samplerate,stopTime,startInd,stopInd
0,kcomplex,E1,0001,0001,830.596676,0.699174,200,831.295850,166119,166259
1,kcomplex,E1,0001,0001,840.981316,0.492156,200,841.473472,168196,168295
2,kcomplex,E1,0001,0001,970.596678,0.578088,200,971.174766,194119,194235
3,kcomplex,E1,0001,0001,1049.772807,0.695268,200,1050.468075,209955,210094
4,kcomplex,E1,0001,0001,1077.231575,0.648396,200,1077.879971,215446,215576


In [130]:
signalsMetadata.head()

,subjectId,file,channel,duration,samplerate,isOriginalSamplerate,database
0,0001,MASS_0001.pkl,C3-CLE,28956.0,200,False,MASS
1,0002,MASS_0002.pkl,C3-CLE,35016.0,200,False,MASS
2,0003,MASS_0003.pkl,C3-CLE,36760.0,200,False,MASS
3,0004,MASS_0004.pkl,C3-CLE,28004.0,200,False,MASS
4,0005,MASS_0005.pkl,C3-CLE,31244.0,200,False,MASS


## define a fixed feature selection to use

In [131]:
featureSelection=loadPickle("EXT_F1_iteration_7.pkl")
featureSelection

,characteristic,bandName,window
21,hjortActivity,sigma,0.5
66,petrosian,broadband,0.5
22,hjortActivity,theta,0.5
79,relativePower,beta1,0.5
29,hjortComplexity,sigma,0.5


## data splits definition
leave-one-subject-out CROSS VALIDATION

In [132]:
""" #define subjects used in the experiment
usedSubjects=np.array(signalsMetadata['subjectId'])
usedSubjects """

" #define subjects used in the experiment\nusedSubjects=np.array(signalsMetadata['subjectId'])\nusedSubjects "

In [133]:
""" valCount=2  #number of signals for validation (selected randomly)

trainSplits=[]
valSplits=[]
testSplits=[]
for i in range(len(usedSubjects)):
    thisTest=usedSubjects[i]
    thisNoTest=np.setdiff1d(usedSubjects,thisTest)
    thisVal=np.random.choice(thisNoTest,size=valCount,replace=False)
    thisTrain=np.setdiff1d(thisNoTest,thisVal)
    trainSplits.append(thisTrain)
    valSplits.append(thisVal)
    testSplits.append(thisTest)

dataSplits=pd.DataFrame({
    'train':trainSplits,
    'val':valSplits,
    'test':testSplits
})

dataSplits """

" valCount=2  #number of signals for validation (selected randomly)\n\ntrainSplits=[]\nvalSplits=[]\ntestSplits=[]\nfor i in range(len(usedSubjects)):\n    thisTest=usedSubjects[i]\n    thisNoTest=np.setdiff1d(usedSubjects,thisTest)\n    thisVal=np.random.choice(thisNoTest,size=valCount,replace=False)\n    thisTrain=np.setdiff1d(thisNoTest,thisVal)\n    trainSplits.append(thisTrain)\n    valSplits.append(thisVal)\n    testSplits.append(thisTest)\n\ndataSplits=pd.DataFrame({\n    'train':trainSplits,\n    'val':valSplits,\n    'test':testSplits\n})\n\ndataSplits "

In [134]:
#by now we used fixed CV data splits instead
dataSplits=loadPickle("dataSplits_MASS_debug")
dataSplits

,train,val,test
0,"[0002, 0003, 0004, 0005, 0007, 0008, 0009, 001...","[0006, 0014]",[0001]
1,"[0001, 0003, 0004, 0005, 0006, 0007, 0008, 000...","[0012, 0010]",[0002]
2,"[0002, 0004, 0005, 0006, 0007, 0008, 0009, 001...","[0001, 0012]",[0003]
3,"[0001, 0003, 0005, 0006, 0008, 0009, 0010, 001...","[0002, 0007]",[0004]
4,"[0001, 0002, 0003, 0004, 0006, 0007, 0008, 000...","[0011, 0016]",[0005]
5,"[0001, 0002, 0003, 0004, 0005, 0007, 0008, 000...","[0011, 0016]",[0006]
6,"[0001, 0002, 0003, 0004, 0005, 0006, 0008, 000...","[0010, 0018]",[0007]
7,"[0001, 0002, 0003, 0004, 0005, 0007, 0009, 001...","[0015, 0006]",[0008]
8,"[0001, 0002, 0003, 0004, 0006, 0008, 0010, 001...","[0005, 0007]",[0009]
9,"[0001, 0002, 0003, 0004, 0005, 0006, 0007, 000...","[0011, 0009]",[0010]


## define annotation criterium
by now we keep the labeler that went trough all the signals

In [135]:
usedAnnotations=annotations[(annotations['type']=='spindle')&(annotations['labelerId']=='0001')]


In [136]:
usedAnnotations[['subjectId','labelerId','duration']].groupby(['subjectId','labelerId']).describe()

duration                                          \
                       count      mean       std       min       25%   
subjectId labelerId                                                    
0001      0001        1044.0  0.822037  0.202792  0.390602  0.675735   
0002      0001        1143.0  0.834582  0.195713  0.414040  0.699173   
0003      0001         143.0  0.690269  0.149652  0.460907  0.585900   
0004      0001         253.0  0.829708  0.224842  0.492157  0.667923   
0005      0001         341.0  0.706104  0.131805  0.425751  0.613243   
0006      0001         150.0  0.761306  0.143614  0.492157  0.679642   
0007      0001         912.0  0.892692  0.226504  0.425751  0.730423   
0008      0001         385.0  0.846882  0.227884  0.441376  0.695267   
0009      0001         814.0  0.923970  0.216783  0.468719  0.773392   
0010      0001         795.0  0.815081  0.190663  0.402321  0.687454   
0011      0001         606.0  0.923982  0.222469  0.503876  0.773384   
0012      0001         709.0  0.836528  0.180787  0.437469  0.706985   
0013      0001         698.0  0.910960  0.245084  0.437477  0.742142   
0014      0001         713.0  0.805534  0.205166  0.406227  0.671829   
0015      0001          97.0  0.739281  0.106173  0.519501  0.671829   
0016      0001         452.0  0.806943  0.210937  0.335915  0.640587   
0017      0001         470.0  0.802766  0.155081  0.437469  0.694296   
0018      0001        1164.0  0.844465  0.185415  0.421852  0.722610   
0019      0001         315.0  0.770511  0.129903  0.453094  0.687454   

                                                   
                          50%       75%       max  
subjectId labelerId                                
0001      0001       0.796822  0.945251  1.839722  
0002      0001       0.816353  0.937439  1.820198  
0003      0001       0.648399  0.775341  1.160080  
0004      0001       0.785103  0.945251  1.675674  
0005      0001       0.699173  0.781197  1.195236  
0006      0001       0.749954  0.847603  1.480377  
0007      0001       0.867134  1.003845  2.042839  
0008      0001       0.800728  0.937439  1.925659  
0009      0001       0.890568  1.027275  1.882690  
0010      0001       0.773392  0.921814  1.847542  
0011      0001       0.890564  1.042900  2.015495  
0012      0001       0.808548  0.956970  1.495995  
0013      0001       0.871036  1.026300  2.117058  
0014      0001       0.773392  0.906197  2.218605  
0015      0001       0.722610  0.808540  1.011650  
0016      0001       0.773384  0.925720  1.679581  
0017      0001       0.781200  0.890570  1.644424  
0018      0001       0.824165  0.941345  1.882690  
0019      0001       0.757767  0.839790  1.359291

## model fitting

In [137]:
def eval_F1(preds, y):
    y = y.get_label()
    score=F1(y,preds)
    return 'anti-F1', 1-score

In [138]:
n_jobs=4 # paralelization parameter for xgboost (keep it 8 as maximum)
learning_rate=0.3
subsample=0.6

models=[]
treeLimits=[]
valF1s=[]
for index, row in dataSplits.iterrows():
    #Features
    trainFeatures=loadFeatureMatrix(row.train,featureSelection,signalsMetadata,samplerate,datapath)
    valFeatures=loadFeatureMatrix(row.val,featureSelection,signalsMetadata,samplerate,datapath)
    #Labels
    trainLabels=loadLabelsVector(row.train,usedAnnotations,signalsMetadata,samplerate)
    valLabels=loadLabelsVector(row.val,usedAnnotations,signalsMetadata,samplerate)
    #Train the models
    print(sum(1-trainLabels)/sum(trainLabels))
    xgb_model = xgb.XGBClassifier(n_jobs=n_jobs, learning_rate=learning_rate, subsample=subsample).fit(trainFeatures, trainLabels, early_stopping_rounds=3, eval_metric=eval_F1,eval_set=[(valFeatures,valLabels)])
    models.append(xgb_model)
    treeLimits.append(xgb_model.best_ntree_limit)
    #val predictions and objective function definition
    valPredictions=xgb_model.predict(valFeatures, ntree_limit=xgb_model.best_ntree_limit)
    valF1s.append(F1(valLabels,valPredictions))


59.59794570100682


c:\Users\Gurdiel\Code\SpinCo\venv\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\Gurdiel\Code\SpinCo\venv\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.44696	validation_0-anti-F1:0.96506
[1]	validation_0-logloss:0.31084	validation_0-anti-F1:0.95084
[2]	validation_0-logloss:0.22510	validation_0-anti-F1:0.93320
[3]	validation_0-logloss:0.16781	validation_0-anti-F1:0.91176
[4]	validation_0-logloss:0.12825	validation_0-anti-F1:0.88626
[5]	validation_0-logloss:0.10026	validation_0-anti-F1:0.85746
[6]	validation_0-logloss:0.08026	validation_0-anti-F1:0.82529
[7]	validation_0-logloss:0.06588	validation_0-anti-F1:0.79097
[8]	validation_0-logloss:0.05537	validation_0-anti-F1:0.75579
[9]	validation_0-logloss:0.04777	validation_0-anti-F1:0.72241
[10]	validation_0-logloss:0.04214	validation_0-anti-F1:0.69077
[11]	validation_0-logloss:0.03803	validation_0-anti-F1:0.66174
[12]	validation_0-logloss:0.03500	validation_0-anti-F1:0.63608
[13]	validation_0-logloss:0.03280	validation_0-anti-F1:0.61570
[14]	validation_0-logloss:0.03117	validation_0-anti-F1:0.59963
[15]	validation_0-logloss:0.03000	validation_0-anti-F1:0.58659
[1

c:\Users\Gurdiel\Code\SpinCo\venv\lib\site-packages\xgboost\core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


64.16151585738564


...

c:\Users\Gurdiel\Code\SpinCo\venv\lib\site-packages\xgboost\core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


60.8605892433981


c:\Users\Gurdiel\Code\SpinCo\venv\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\Gurdiel\Code\SpinCo\venv\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.44901	validation_0-anti-F1:0.93836
[1]	validation_0-logloss:0.31473	validation_0-anti-F1:0.92151
[2]	validation_0-logloss:0.23065	validation_0-anti-F1:0.90095
[3]	validation_0-logloss:0.17478	validation_0-anti-F1:0.87631
[4]	validation_0-logloss:0.13641	validation_0-anti-F1:0.84752
[5]	validation_0-logloss:0.10955	validation_0-anti-F1:0.81558
[6]	validation_0-logloss:0.09047	validation_0-anti-F1:0.78061
[7]	validation_0-logloss:0.07677	validation_0-anti-F1:0.74453
[8]	validation_0-logloss:0.06689	validation_0-anti-F1:0.70919
[9]	validation_0-logloss:0.05976	validation_0-anti-F1:0.67507
[10]	validation_0-logloss:0.05456	validation_0-anti-F1:0.64485
[11]	validation_0-logloss:0.05080	validation_0-anti-F1:0.61743
[12]	validation_0-logloss:0.04804	validation_0-anti-F1:0.59437
[13]	validation_0-logloss:0.04603	validation_0-anti-F1:0.57575
[14]	validation_0-logloss:0.04456	validation_0-anti-F1:0.56041
[15]	validation_0-logloss:0.04351	validation_0-anti-F1:0.54814
[1

c:\Users\Gurdiel\Code\SpinCo\venv\lib\site-packages\xgboost\core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


In [139]:
dataSplits["model"]=models
dataSplits["treeLimit"]=treeLimits
dataSplits["valF1"]=valF1s

In [140]:
dataSplits

,train,val,test,model,treeLimit,valF1
0,"[0002, 0003, 0004, 0005, 0007, 0008, 0009, 001...","[0006, 0014]",[0001],"XGBClassifier(base_score=0.5, booster='gbtree'...",50,0.623415
1,"[0001, 0003, 0004, 0005, 0006, 0007, 0008, 000...","[0012, 0010]",[0002],"XGBClassifier(base_score=0.5, booster='gbtree'...",54,0.560644
2,"[0002, 0004, 0005, 0006, 0007, 0008, 0009, 001...","[0001, 0012]",[0003],"XGBClassifier(base_score=0.5, booster='gbtree'...",61,0.373103
3,"[0001, 0003, 0005, 0006, 0008, 0009, 0010, 001...","[0002, 0007]",[0004],"XGBClassifier(base_score=0.5, booster='gbtree'...",61,0.619176
4,"[0001, 0002, 0003, 0004, 0006, 0007, 0008, 000...","[0011, 0016]",[0005],"XGBClassifier(base_score=0.5, booster='gbtree'...",55,0.641606
5,"[0001, 0002, 0003, 0004, 0005, 0007, 0008, 000...","[0011, 0016]",[0006],"XGBClassifier(base_score=0.5, booster='gbtree'...",54,0.642723
6,"[0001, 0002, 0003, 0004, 0005, 0006, 0008, 000...","[0010, 0018]",[0007],"XGBClassifier(base_score=0.5, booster='gbtree'...",46,0.592171
7,"[0001, 0002, 0003, 0004, 0005, 0007, 0009, 001...","[0015, 0006]",[0008],"XGBClassifier(base_score=0.5, booster='gbtree'...",80,0.432130
8,"[0001, 0002, 0003, 0004, 0006, 0008, 0010, 001...","[0005, 0007]",[0009],"XGBClassifier(base_score=0.5, booster='gbtree'...",46,0.594039
9,"[0001, 0002, 0003, 0004, 0005, 0006, 0007, 000...","[0011, 0009]",[0010],"XGBClassifier(base_score=0.5, booster='gbtree'...",55,0.564562


In [142]:
dumpPickle("Experiment002_CV_"+str(time.time())+".pkl",[dataSplits,featureSelection])